In [1]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
embeddings

/tmp/ipykernel_64154/3402322883.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/home/hammadali08/PycharmProjects/RAG/.venv/lib64/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

### FOR PDFs

Also added the Embeddings

In [3]:
## Data Ingestion
from typing import List, Dict, Optional
from langchain_community.document_loaders import PyPDFLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings


class SmartPDFProcessor:
    """Advanced PDF processing using Semantic Chunking with metadata enhancement"""

    def __init__(self, model_name: str = "sentence-transformers/all-MiniLM-L6-v2"):
        # Initialize the embedding model for semantic chunking
        self.embedding_model = HuggingFaceEmbeddings(model_name=model_name)
        self.text_splitter = SemanticChunker(self.embedding_model)

    def process_pdf(
        self,
        pdf_path: str,
        custom_metadata: Optional[Dict[str, str]] = None
    ) -> List[Document]:
        """Process PDF with semantic chunking and metadata enhancement"""

        # Step 1: Load PDF
        loader = PyPDFLoader(pdf_path)
        pages = loader.load()

        processed_chunks = []

        for page_num, page in enumerate(pages):
            cleaned_text = self._clean_text(page.page_content)

            # Skip nearly empty pages
            if len(cleaned_text.strip()) < 50:
                continue

            # Step 2: Metadata collection
            metadata = {
                "page": page_num + 1,
                "total_pages": len(pages),
                "chunk_method": "SemanticChunker",
                "char_count": len(cleaned_text),
            }

            # Add PDF metadata if present
            if page.metadata:
                metadata.update(page.metadata)

            # Add user-provided metadata if present
            if custom_metadata:
                metadata.update(custom_metadata)

            # Step 3: Perform semantic chunking
            chunks = self.text_splitter.create_documents(
                texts=[cleaned_text],
                metadatas=[metadata]
            )

            processed_chunks.extend(chunks)

        return processed_chunks

    def _clean_text(self, text: str) -> str:
        """Clean up common PDF text issues"""
        text = " ".join(text.split())
        text = text.replace("ﬁ", "fi").replace("ﬂ", "fl")
        return text


In [4]:
pdfchucker=SmartPDFProcessor()
Processed_chunks=pdfchucker.process_pdf('/home/hammadali08/Downloads/CS/Scheme of Studies of BS Computer Science.pdf')

In [5]:
print(Processed_chunks[111].page_content)

 Understand and use linear transformations.  Perform LU decompositions.  Evaluate and apply eigenvectors and eigen values. Learning Outcomes At the completion of the course, the student will be able to:  Demonstrate the concept ofmatrices, determinant, linear transformations, linear system  Gain the ability to apply the gained knowledge in different field of computer science Recommende d Text Book  Linear Algebra, Lipschutz, S. and Lipson, M. (2009), New York, McGraw-Hill  Elementary Linear Algebra by Howard Anton  Linear Algebra and its Applications by Gibert Strang Course Content Vectors in Rn and Cn: Vectors in Rn, Linear combination of Vectors, Dot (Inner) Product, Angle Between Vectors, Projections, Cauchy -Schwarz Inequality, Minkowski‘s Inequality, Hyperplanes, Lines, VectorsinR3 (Spatial Vectors), Cross Product, Lagrange‘s Identity Matrix Operations: Matrices, Trace, Powers of Matrices, Polynomials in Matrices, Invertible Matrices, Orthogonal Matrices, Normal Matrices, 

In [6]:
from langchain_community.vectorstores import Chroma
persist_directory = 'chroma-db'
vectordb = Chroma.from_documents(documents=Processed_chunks, 
                                 embedding=embeddings,
                                 persist_directory=persist_directory)
vectordb.persist()

/tmp/ipykernel_64154/3128432432.py:6: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [7]:
vectordb.similarity_search('Aritificial Intelligence')

[Document(metadata={'char_count': 2411, 'author': 'Farrah Aslam', 'producer': 'Microsoft® Word 2010', 'total_pages': 111, 'page_label': '28', 'creator': 'Microsoft® Word 2010', 'moddate': '2022-10-13T14:21:15+05:00', 'page': 27, 'title': 'BS CS Curriculum', 'source': '/home/hammadali08/Downloads/CS/Scheme of Studies of BS Computer Science.pdf', 'keywords': 'ver 1', 'chunk_method': 'SemanticChunker', 'creationdate': '2022-06-28T14:42:26+05:00'}, page_content='Bender). \uf0b7 Principals of Artificial Intelligence and Expert Systems Development. (David W. Rolston) \uf0b7 Nils J Nilson, Artificial Intelligence – A New Synthesis, Morgan Kaufman Publishers, Elsevier, USA. \uf0b7 Patrick Henry Winston, Artificial Intelligence, Third Edition, Pearson Education Series in AI. Course Content Introduction: History, Applications and Future, Agents and environments, Rationality, Agent Types, Propositional Logic, Predicate Calculus Problem solving through uninformed search methods; State Space Search

In [8]:
## GROQ LLM API
import os
os.environ['api_key']=os.getenv('api_key')

In [9]:
from langchain_groq import ChatGroq
llm = ChatGroq(model_name="openai/gpt-oss-120b", api_key=os.environ['api_key'])

In [10]:
from langchain.chat_models.base import init_chat_model
llm=init_chat_model("groq:openai/gpt-oss-120b",api_key=os.environ['api_key'], temperature=0.2)

In [11]:
print(llm.invoke('AI'))

content="Hello! How can I assist you with AI today? Whether you have questions about how it works, need help with a project, or just want to chat about the latest developments, I'm here to help. Let me know what you're interested in!" additional_kwargs={'reasoning_content': 'The user just wrote "AI". Likely they want a response about AI? Could be a prompt to discuss AI. We need to respond appropriately. Since no specific request, we can ask clarifying question or give a brief overview. Probably respond with a friendly greeting and ask how can help.'} response_metadata={'token_usage': {'completion_tokens': 118, 'prompt_tokens': 72, 'total_tokens': 190, 'completion_time': 0.238739602, 'prompt_time': 0.007349869, 'queue_time': 0.049821476, 'total_time': 0.246089471}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_3a688838c3', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--7f73e1d2-2e7f-4fc4-9439-61679cf39bd3-0' usage_metadata={'input_token

## RAG CHAIN

### Ensemble Retriever Method

In [12]:
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever

In [13]:
sparce_retriever = BM25Retriever.from_documents(Processed_chunks)
sparce_retriever.k=10

In [34]:
## Convert vector store to RAG chain
retriever = vectordb.as_retriever(
    search_kwargs={"k": 5})

In [35]:
Combined_retriever = EnsembleRetriever(
    retrievers=[retriever, sparce_retriever],
    weights=[0.3, 0.7]
)

## Reranking Retriver

In [36]:
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers import ContextualCompressionRetriever

In [37]:
cross_encoder = HuggingFaceCrossEncoder(model_name="cross-encoder/ms-marco-MiniLM-L-2-v2")

# Wrap it as a reranker
reranker = CrossEncoderReranker(model=cross_encoder, top_n=10)

# Combine with your base retriever
reranking_retriever = ContextualCompressionRetriever(
    base_compressor=reranker,
    base_retriever=Combined_retriever
)

In [38]:
reranking_retriever.get_relevant_documents("what are the subjects in 5th semester?")

[Document(metadata={'page': 67, 'total_pages': 111, 'chunk_method': 'SemanticChunker', 'char_count': 2666, 'producer': 'Microsoft® Word 2010', 'creator': 'Microsoft® Word 2010', 'creationdate': '2022-06-28T14:42:26+05:00', 'moddate': '2022-10-13T14:21:15+05:00', 'title': 'BS CS Curriculum', 'author': 'Farrah Aslam', 'keywords': 'ver 1', 'source': '/home/hammadali08/Downloads/CS/Scheme of Studies of BS Computer Science.pdf', 'page_label': '68'}, page_content='5th Edition, New York: Longman \uf0b7 Psychology and Life, Zimbardo & Gerrig (14th Edition), Harper Collins College Publishers. Course Content Introduction to Psychology, Historical Roots of Modern Psychology, Schools of Thought, Perspective/Model/Approach, Psychodynamic Approach/ Model, Behavioral Approach, The Humanistic Approach and the Cognitive Approach, Research Methods in Psychology, Physi cal Development and Nature Nurture, Cognitive Development, Nervous System, Endocrine System, Sensation, Hearing (Audition) and Balance, P

In [39]:
from langchain.chains import RetrievalQA

rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=reranking_retriever,
    return_source_documents=True
)

response = rag_chain.invoke({"query": "what are the subjects we will study in Semester – VII?"})
print(response["result"])

**Semester VII – Courses**

| Code (tentative) | Course Title                              | Credit Hours |
|------------------|-------------------------------------------|--------------|
| **COMP4113**     | Compiler Construction                     | 3 (3 + 0) |
| **COMP4114**     | Artificial Intelligence                   | 4 (3 + 1) |
| **COMPXXXX**     | CS Elective – IV *(code to be assigned after Academic Council approval)* | 3 (3 + 0) |
| **COMP4115**     | Final Year Project                        | 3 (0 + 3) |

**Total:** 13 credit hours (9 theory + 4 practical).  

These are the subjects you will study in Semester VII.


In [40]:
# Creating a Prompt template
system_prompt = """
You are an assistant that answers questions about the University Scheme of Studies.

Rules:
1. Use only the provided context to answer. Never guess or use outside knowledge.
2. If the answer is not found in the context, reply exactly with: "Sorry! I don't know."
3. Keep responses concise, factual, and directly relevant to the question.
4. If asked for a course outline or syllabus, return the complete course content from the context.

Context:
{context}
"""




prompt=ChatPromptTemplate([
    
    ('system',system_prompt),
    ('human','{input}')
    ] )

In [41]:
## Creating Document Chain : Will combine the chat_prompt & LLM
document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nYou are an assistant that answers questions about the University Scheme of Studies.\n\nRules:\n1. Use only the provided context to answer. Never guess or use outside knowledge.\n2. If the answer is not found in the context, reply exactly with: "Sorry! I don\'t know."\n3. Keep responses concise, factual, and directly relevant to the question.\n4. If asked for a course outline or syllabus, return the complete course content from the context.\n\nContext:\n{context}\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types=

In [42]:
### Creating the Final RAG chain where retriver adds into the chain
rag_chain=create_retrieval_chain(reranking_retriever,document_chain)
rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | ContextualCompressionRetriever(base_compressor=CrossEncoderReranker(model=HuggingFaceCrossEncoder(client=CrossEncoder(
               (model): BertForSequenceClassification(
                 (bert): BertModel(
                   (embeddings): BertEmbeddings(
                     (word_embeddings): Embedding(30522, 384, padding_idx=0)
                     (position_embeddings): Embedding(512, 384)
                     (token_type_embeddings): Embedding(2, 384)
                     (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
                     (dropout): Dropout(p=0.1, inplace=False)
                   )
                   (encoder): BertEncoder(
                     (layer): ModuleList(
                       (0-1): 2 x BertLayer(
                         (attention): BertAttention(
                           (self): BertSdpaSelfAttent

In [43]:
response=rag_chain.invoke({"input":"what are the names of subjects in which we will study Neural Networks in BS Computer Science?"})
response

{'input': 'what are the names of subjects in which we will study Neural Networks in BS Computer Science?',
 'context': [Document(metadata={'producer': 'Microsoft® Word 2010', 'keywords': 'ver 1', 'chunk_method': 'SemanticChunker', 'total_pages': 111, 'title': 'BS CS Curriculum', 'author': 'Farrah Aslam', 'moddate': '2022-10-13T14:21:15+05:00', 'creationdate': '2022-06-28T14:42:26+05:00', 'char_count': 3227, 'page': 5, 'creator': 'Microsoft® Word 2010', 'source': '/home/hammadali08/Downloads/CS/Scheme of Studies of BS Computer Science.pdf', 'page_label': '6'}, page_content='5 1. Curriculum for Bachelor of Science in Computer Science (BSCS) Program Rationale Computer science (CS) is the study of computers and algorithmic processes, including their principles, their hardware and software designs, their applications, and their impact on society [Ref ACM Task Force 2006]. Computer Science is the study o f computers and computational systems encompassing its theoretical study, design, develo

# Agentic RAG

In [44]:
from langchain.chains import RetrievalQA
from langchain.agents import initialize_agent, Tool

retrieval_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=reranking_retriever,
    chain_type="stuff",

    return_source_documents=False,
)

# -------------------------------
# 5. Register the retriever as a Tool
# -------------------------------
tools = [
    Tool(
        name="University Document Retriever",
        func=retrieval_chain.run,
        description='''Use this tool to retrieve and answer questions about university courses, 
                        schemes of study, credit hours, and subject outlines using official documentation.'''
    )
]

# -------------------------------
# 6. Create a FAST Agent
# -------------------------------
agent = initialize_agent(
    tools,
    llm=llm,
    agent_type="zero-shot-react-description",  # lightest agent type
    verbose=True,
    handle_parsing_errors=True
)

In [45]:
response=agent.run({"input":"what are the selected subjects in semester 7"})
response



> Entering new AgentExecutor chain...
Question: what are the selected subjects in semester 7
Thought: I need to retrieve the official university document that lists the selected subjects for semester 7.
Action: University Document Retriever
Action Input: "semester 7 selected subjects"
Observation: I’m sorry, but the information you provided does not include a detailed list of the courses that are scheduled for Semester 7 of the BS Computer Science program. If you have a more complete semester‑wise study plan or a course catalog that shows the Semester 7 offerings, I can help you interpret it. Otherwise, I don’t have enough data to list the selected subjects for that semester.
Thought:Question: what are the selected subjects in semester 7  
Thought: I need to locate the official university document that lists the subjects offered in Semester 7. The previous query was too vague, so I’ll try a broader search term that might match a semester‑wise study plan or course catalog.  
Action: U

'The selected subjects for Semester\u202f7 are:\n\n| Course Code | Course Title                | Credit Hours* |\n|-------------|-----------------------------|---------------|\n| COMP4113    | Compiler Construction       | 3\u202f(3\u202f+\u202f0) |\n| COMP4114    | Artificial Intelligence     | 4\u202f(3\u202f+\u202f1) |\n| COMPXXXX    | CS Elective –\u202fIV            | 3\u202f(3\u202f+\u202f0) |\n| COMP4115    | Final Year Project          | 3\u202f(0\u202f+\u202f3) |\n\n\\*Credit hours are shown as **theory\u202f+\u202flab/project** components.\n\n**Total credit load:** 13 credits (9 theory credits + 4 project credits).'

In [46]:
mmr_retriever = vectordb.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "fetch_k": 20, "lambda_mult": 0.5}
)

In [47]:
mmr_retriever.get_relevant_documents("What is the outline for AI?")

[Document(metadata={'title': 'BS CS Curriculum', 'page_label': '28', 'source': '/home/hammadali08/Downloads/CS/Scheme of Studies of BS Computer Science.pdf', 'keywords': 'ver 1', 'creator': 'Microsoft® Word 2010', 'author': 'Farrah Aslam', 'total_pages': 111, 'creationdate': '2022-06-28T14:42:26+05:00', 'moddate': '2022-10-13T14:21:15+05:00', 'producer': 'Microsoft® Word 2010', 'chunk_method': 'SemanticChunker', 'char_count': 2411, 'page': 27}, page_content='Bender). \uf0b7 Principals of Artificial Intelligence and Expert Systems Development. (David W. Rolston) \uf0b7 Nils J Nilson, Artificial Intelligence – A New Synthesis, Morgan Kaufman Publishers, Elsevier, USA. \uf0b7 Patrick Henry Winston, Artificial Intelligence, Third Edition, Pearson Education Series in AI. Course Content Introduction: History, Applications and Future, Agents and environments, Rationality, Agent Types, Propositional Logic, Predicate Calculus Problem solving through uninformed search methods; State Space Search

In [48]:
reranker = CrossEncoderReranker(model=cross_encoder, top_n=10)

# Combine with your base retriever
reranking_retriever_mmr = ContextualCompressionRetriever(
    base_compressor=reranker,
    base_retriever=mmr_retriever
)

In [51]:
reranking_retriever_mmr.get_relevant_documents("What are the key points in outline for AI that i will need to focus more on?")

[Document(metadata={'page_label': '3', 'keywords': 'ver 1', 'total_pages': 111, 'producer': 'Microsoft® Word 2010', 'char_count': 4693, 'page': 2, 'moddate': '2022-10-13T14:21:15+05:00', 'source': '/home/hammadali08/Downloads/CS/Scheme of Studies of BS Computer Science.pdf', 'creationdate': '2022-06-28T14:42:26+05:00', 'chunk_method': 'SemanticChunker', 'creator': 'Microsoft® Word 2010', 'title': 'BS CS Curriculum', 'author': 'Farrah Aslam'}, page_content='2 5. Computer Science Computing CORE Courses ................................................................................... 30 Course Outlines for Computer Science Core Courses ............................................................................ 31 Programming Fundamentals ............................................................................................................... 31 Object Oriented Programming ............................................................................................................ 

In [50]:
document_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(reranking_retriever_mmr,document_chain)

response = rag_chain.invoke({"input": "What are the key points in outline for AI that i will need to focus more on?"})
print(response['answer'])

**Key points in the AI course outline to focus on**

- **Introduction**
  - History, applications and future of AI  
- **Agents and Environments**
  - Rationality and different agent types  
- **Knowledge Representation**
  - Propositional logic  
  - Predicate calculus  
- **Problem‑solving Techniques**
  - Uninformed (blind) search methods  
    - State‑space search  
    - Depth‑first search  
    - Breadth‑first search  
    - Comparisons of methods  
  - Informed (heuristic) search methods  
- **Expert Systems**
  - Design and implementation basics  

These topics form the core structure of the AI course and are the areas you should prioritize in your study.


# Query Ehnancmenet

In [31]:
from langchain.embeddings import HuggingFaceEmbeddings

# Step 1: Initialize models
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en")

# Step 2: Function to enhance query
def enhance_query(query: str):
    # 1. Generate embedding-based semantic enhancement
    embedded_query = embedding_model.embed_query(query)

    # 2. Use LLM to refine the query in natural language
    refined_query = llm.invoke(
        f"""You are assisting a university document search system.
    Take the user's query and rewrite it to make it more detailed, specific, and context-aware
    for retrieving relevant information from university academic documents.

    Focus on improving clarity, adding missing academic context (like department, semester, or course structure),
    and expanding abbreviations or vague terms if needed.

    User Query: {query}

    Return only the improved version of the query, nothing else."""
    )

    return refined_query



In [32]:
# Example usage
query = "What are the course subjects in the semester 7 of BS Computer Science and the outline of those?"
enhanced_query = enhance_query(query)
print("Enhanced Query:\n", enhanced_query.content)

Enhanced Query:
 What are the specific course subjects offered in Semester 7 of the Bachelor of Science in Computer Science program (BS Computer Science) within the Department of Computer Science, and can you provide the detailed syllabus or outline for each of these courses, including topics covered, credit hours, and any prerequisite requirements?
